### Template for enabling existing code to run efficiently on NERSC

In [1]:
%load_ext jupyter_ai_magics

scalesci_prompt = '''You are an expert in optimizing scientific computations on HPC systems. 
You will help this scientist take their existing code and turn it into a Jupyter notebook 
utilizing dask with improved performance (faster, more interactive). 
Start by providing a short summary of the computation (DO NOT list out all the steps), followed by 
a short list of improvement suggestions for the following code.'''

rewrite_prompt = '''Incorporate the suggestions and modify the existing code.'''

def read_file_to_string(file_path):
    try:
        with open(file_path, 'r') as file:
            return file.read()
    except FileNotFoundError:
        print(f"{file_path} not found.")
        return None

#### Fill in the file name of the code that you would like help with

In [2]:
file_name = "/Users/kberket/src/AEW_Energetics/bin/WRF_EKE_NERSC_Ex.py"
file_contents = read_file_to_string(file_name)

In [3]:
%%ai chatgpt
{scalesci_prompt}
--
{file_contents}
--

The computation in the given code calculates the eddy kinetic energy (EKE) and plots its meridional distribution using WRF data. It performs several operations including reading netCDF files, applying temporal and spatial filtering, averaging, integrating, and plotting.

Improvement suggestions for the code:
1. Import only the required functions/modules from the libraries (`from x import y` instead of `import x`).
2. Use Dask to parallelize and optimize the computation.
3. Use Dask's lazy evaluation to delay the execution of operations until necessary.
4. Avoid unnecessary variable assignments and deletions.
5. Use Dask's distributed scheduler for distributed computing on HPC systems.
6. Use NumPy's vectorized operations instead of looping.
7. Utilize Dask's chunking capabilities for handling large datasets.
8. Optimize filtering operations using more efficient algorithms or libraries.
9. Avoid unnecessary imports that are not used in the code.
10. Use implicit division (`from __future__ import division`) instead of explicit division (`/`).
11. Remove unnecessary print statements.

Note: The modifications will depend on the specifics of the HPC system and the dataset being used.

In [4]:
%%ai chatgpt -f code
{rewrite_prompt}

In [ ]:
from __future__ import division
import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.backends.backend_pdf import PdfPages
import scipy.ndimage as ndimage
import dask.array as da

import wrf as wrf
from scipy import signal


def lat_lon(data):
    lat = wrf.getvar(data, "lat")
    lon = wrf.getvar(data, "lon")
    lat_crop = lat.values[lat_index_south:lat_index_north+1, lon_index_west:lon_index_east+1]
    lon_crop = lon.values[lat_index_south:lat_index_north+1, lon_index_west:lon_index_east+1]
    return lat_crop, lon_crop

    
def butter_bandpass_filter(variable, fs):
    order = 6
    big_period_day = 5.0
    small_period_day = 3.0
    big_period_hr = big_period_day * 24.0
    small_period_hr = small_period_day * 24.0
    nyq = .5 * fs
    low_frequency = (1 / big_period_hr) / nyq
    high_frequency = (1 / small_period_hr) / nyq
    b, a = signal.butter(order, [low_frequency, high_frequency], btype='bandpass')
    filtered_variable = signal.lfilter(b, a, variable, axis=0)
    return filtered_variable


def main():
    scenario_type = 'Historical'
    g = 9.8
    
    lat, lon, lat_index_north, lat_index_south, lon_index_west, lon_index_east = lat_lon()

    eke_list = []
    total_eke_list = []
    
    for year in range(2001, 2011):
        print('Year =', year)
        file_location = '/global/cscratch1/sd/ebercosh/WRF_TCM/' + scenario_type + '/' + str(year) + '/'

        u_data = xr.open_dataset(file_location + 'ua_' + scenario_type + '_' + str(year) + '.nc')
        v_data = xr.open_dataset(file_location + 'va_' + scenario_type + '_' + str(year) + '.nc')
        u_4d_full = u_data.ua
        v_4d_full = v_data.va
        print(u_4d_full.shape)
        u_4d = u_4d_full[:-120, :, :, :]
        v_4d = v_4d_full[:-120, :, :, :]
        print(u_4d.shape)
        
        fs = 1/6
        u_temp_filt = da.from_array(u_4d, chunks=(240, "auto", "auto", "auto"))
        u_temp_filt = da.map_blocks(butter_bandpass_filter, u_temp_filt, fs, dtype=u_4d.dtype)
        v_temp_filt = da.from_array(v_4d, chunks=(240, "auto", "auto", "auto"))
        v_temp_filt = da.map_blocks(butter_bandpass_filter, v_temp_filt, fs, dtype=v_4d.dtype)
        u_temp_filt, v_temp_filt = da.compute(u_temp_filt, v_temp_filt)

        uu = da.square(u_temp_filt)
        vv = da.square(v_temp_filt)

        uu_3d = da.mean(uu, axis=0)
        vv_3d = da.mean(vv, axis=0)
        
        uu_vv_3d = uu_3d + vv_3d

        uu_vv_2d = da.mean(uu_vv_3d[:, :, lon_index_west:lon_index_east+1], axis=2)

        eke_2d = 0.5 * uu_vv_2d
        eke_list.append(eke_2d)

        eke_1d = 0.5 * da.mean(uu_vv_2d[:, lat_index_south:lat_index_north+1] / g, axis=1)

        total_eke = da.trapz(eke_1d, dx=5000., axis=0)
        total_eke_list.append(total_eke)

    total_eke_avg = da.mean(da.stack(total_eke_list), axis=0)
    print('10 year total EKE for', scenario_type, '=', total_eke_avg.compute(), 'J/m^2')

    eke_avg = da.mean(da.stack(eke_list), axis=0)
    print(eke_avg.shape)

    eke_smooth = da.from_array(ndimage.gaussian_filter(eke_avg.compute(), sigma=0.9, order=0))
    
    fig, ax = plt.subplots()
    clevels = np.linspace(-8, 8, 33)
    cmap = plt.cm.PuOr
    units = '$\mathrm{m}^{2}$ $\mathrm{s}^{-2}$'
    p_levels = np.linspace(1000, 100, 19)
    X, Y = np.meshgrid(lat[:, 0], p_levels)
    plt.xlabel("Latitude")
    plt.ylabel("Pressure (hPa)")
    contours_fill = plt.contourf(X, Y, eke_avg.compute(), cmap=cmap, levels=clevels, extend="both")
    ax.set_xlim(-5, 25)
    plt.minorticks_on()
    plt.gca().invert_yaxis()
    cbar = plt.colorbar(contours_fill, shrink=.58, orientation='horizontal')
    ax.set_aspect(1./ax.get_data_ratio())
    fig.savefig('WRF_TCM_M-O_2001-2010avg_' + scenario_type + '_EKE.pdf', bbox_inches='tight')


if __name__ == '__main__':
    main()


In [ ]:
%%ai chatgpt --reset
reset the chat history